<a href="https://colab.research.google.com/github/sarahaptumich/capstone/blob/shaptonstall/Brace_Classifier_Hyperparameter.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# BRACE Breakdancing
We will use an annotated breakdancing dataset puclish in this GitHub: https://github.com/dmoltisanti/brace.git This repository contains the dataset published with the ECCV 2022 paper "BRACE: The Breakdancing Competition Dataset for Dance Motion Synthesis". We will attempt to train a model that will predict 3 distint breakdancing movements (toprock, powermove and footwork)

# Hyper-paramether Tunning.

In [ ]:
#Mount google drive
from google.colab import drive
drive.mount("/content/drive")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
pip install keras-tuner

In [ ]:
import pickle
import pandas as pd
import numpy as np
from pathlib import Path
import matplotlib.pyplot as plt
from IPython.display import Image
import random as python_random
import importlib.util
import sys
import os

In [ ]:
sys.path.insert(0,'./brace/utils/')
# import keypoints_angles
from keypoints_angles import  angles_distance_segments

In [ ]:
sys.path.insert(0,'./brace/utils/clasification_models.py')
import clasification_models

In [ ]:
import tensorflow as tf
import datetime
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical
from keras_tuner import RandomSearch
from tensorflow.keras.callbacks import TensorBoard, ModelCheckpoint
from tensorflow.keras.models import load_model
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler, OneHotEncoder

Set random seed for reproductivity

In [ ]:
def reset_random_seeds():
   np.random.seed(42)
   python_random.seed(42)
   tf.random.set_seed(42)
   import os
   os.environ['PYTHONHASHSEED'] = '0'

reset_random_seeds()

# Disable all GPUs
tf.config.set_visible_devices([], 'GPU')


Load processed dataClips, labels and names.
See Brace_Dance_Classifier is you do not have the pre-processed dataclips

In [ ]:
#load segment
segments= pd.read_csv(Path('./brace/annotations/segments.csv'))
# Load the TensorDataset
with open('./brace/dataset/dataClips', 'rb') as f:
    dataClips = pickle.load(f)

# Load the training Subset
with open('./brace/dataset/labels', 'rb') as f:
    labels = pickle.load(f)

# Load the testing Subset
with open('./brace/dataset/names', 'rb') as f:
    names = pickle.load(f)

#encode labels from 0-2
unique_categories = list(set(labels))
category_to_int = {category: index for index, category in enumerate(unique_categories)}
encoded_categories = [category_to_int[category] for category in labels]
#get flatted clips
flattened_clips = [clip.reshape(clip.shape[0], -1) for clip in dataClips]

In [ ]:
features= segments.copy()
features['uid']= pd.Categorical(features['uid'], categories=names, ordered=True)
features= features.sort_values(by='uid')
features['frames']= features['end_frame']- features['start_frame']

In [ ]:
sequence_lengths = [len(seq) for seq in flattened_clips]
maxlen = np.percentile(sequence_lengths, 90)
print("Max Lenght 90th percentile:",maxlen)

Max Lenght 90th percentile: 471.9000000000001


### Hyper parameter tuning baseline model
Additional information on parameter tunning with Keras can be found her: https://keras.io/guides/keras_tuner/getting_started/

Tensor Board:
* https://www.tensorflow.org/tensorboard/get_started
* https://www.youtube.com/watch?v=PG4XGqUeYnM

## We will perform hyperparameter tuning on our model with DataClips only

In [ ]:
#prepare data by adding padding and onehot code the categories.
padded_clips = pad_sequences(flattened_clips, padding='pre', truncating='pre', maxlen=int(maxlen), dtype='float32')
X_train, X_test, y_train, y_test = train_test_split(
    padded_clips, encoded_categories, test_size=0.20, random_state=42)

y_train_onehot = to_categorical(y_train, num_classes=3)
y_test_onehot = to_categorical(y_test, num_classes=3)

In [ ]:
input_shape = (X_train.shape[1], X_train.shape[2])
num_classes = 3

In [ ]:
modelDataClips=clasification_models.MdataClips(input_shape=input_shape, num_classes=num_classes)

In [ ]:
project_name= 'modelDataClips_tuning'
tuner = RandomSearch(
    modelDataClips,
    objective='val_accuracy',
    max_trials=10,
    executions_per_trial=1,
    directory='tuner_results',
    project_name=project_name
)

callbacks = [
        TensorBoard(log_dir=os.path.join('logs', project_name)),
        ModelCheckpoint(f'best_model_{project_name}.h5', monitor='val_accuracy', save_best_only=True, verbose=1)
    ]

tuner.search(X_train, y_train_onehot, epochs=10, validation_split=0.2, callbacks=callbacks)

best_model = tuner.get_best_models(num_models=1)[0]
best_model.save(f'best_model_{project_name}.h5')

Trial 10 Complete [00h 12m 25s]
val_accuracy: 0.6866359710693359

Best val_accuracy So Far: 0.8894008994102478
Total elapsed time: 01h 33m 11s


In [ ]:
loaded_model = load_model("./best_model_modelDataClips_tuning.h5")
test_loss, test_accuracy = loaded_model.evaluate(X_test, y_test_onehot)
print(f"Test Loss: {test_loss}, Test Accuracy: {test_accuracy}")

9/9 [==============================] - 6s 618ms/step - loss: 0.3594 - accuracy: 0.8598
Test Loss: 0.35940009355545044, Test Accuracy: 0.8597785830497742


In [ ]:
# # Load the TensorBoard notebook extension
# %load_ext tensorboard
# %tensorboard --logdir logs

### Hyperparamether tuning on the feature and DataClips model

In [ ]:
# reset the logs from previous

!rm -rf /logs/

In [ ]:
##Prepocess additional features to augment our model
reset_random_seeds()
preprocessor = ColumnTransformer(
    transformers=[
        ('seq', OneHotEncoder(), ['seq_idx']),
        ('frames', OneHotEncoder(), ['frames'])
    ]
)

features_preprocessed = preprocessor.fit_transform(features[['seq_idx', 'frames']])
features_preprocessed = features_preprocessed.toarray()

In [ ]:
padded_clips = pad_sequences(flattened_clips, padding='pre', truncating='pre', maxlen=int(maxlen), dtype='float32')

X_train_clips, X_test_clips, X_train_features, X_test_features, y_train, y_test = train_test_split(
    padded_clips, features_preprocessed, encoded_categories, test_size=0.2, random_state=42)


y_train_onehot = to_categorical(y_train, num_classes=3)
y_test_onehot = to_categorical(y_test, num_classes=3)



In [ ]:
reset_random_seeds()
input_shape_clips = (X_train_clips.shape[1], X_train_clips.shape[2])
input_shape_features = (X_train_features.shape[1],)


featureDataClips=clasification_models.FeatureModel(input_shape_clips, input_shape_features, num_classes=3)

project_name= 'features_tuning'
tuner = RandomSearch(
    featureDataClips,
    objective='val_accuracy',
    max_trials=10,
    directory='tuner_results',
    project_name= project_name
)

callbacks = [
        TensorBoard(log_dir=os.path.join('logs', project_name)),
        ModelCheckpoint(f'best_model_{project_name}.h5', monitor='val_accuracy', save_best_only=True, verbose=1)
    ]
tuner.search([X_train_clips, X_train_features], y_train_onehot, epochs=10, validation_split=0.2, callbacks=callbacks)

best_model = tuner.get_best_models(num_models=1)[0]
best_model.save(f'best_model_{project_name}.h5')

Trial 10 Complete [00h 01m 27s]
val_accuracy: 0.8064516186714172

Best val_accuracy So Far: 0.8341013789176941
Total elapsed time: 00h 15m 12s


In [ ]:
loaded_model = load_model("./best_model_features_tuning.h5")
test_loss, test_accuracy = loaded_model.evaluate([X_test_clips, X_test_features], y_test_onehot)
print(f"Test Loss: {test_loss}, Test Accuracy: {test_accuracy}")

9/9 [==============================] - 1s 60ms/step - loss: 0.4346 - accuracy: 0.8266
Test Loss: 0.4345933496952057, Test Accuracy: 0.8265682458877563


### Hyperparameter tune a model that uset DataClip, features and distance and angles

In [ ]:
# reset the logs from previous
!rm -rf /logs/

In [ ]:
hip_knee_ankle_angles= angles_distance_segments(flattened_clips)
# Padding sequences
padded_clips = pad_sequences(flattened_clips, padding='pre', truncating='pre', maxlen=int(maxlen), dtype='float32')
padded_angles = pad_sequences(hip_knee_ankle_angles, padding='pre', truncating='pre', maxlen=int(maxlen), dtype='float32')

#Scaler
n_samples = padded_angles.shape[0]
n_features = padded_angles.shape[1] * padded_angles.shape[2]
flattened_angles = padded_angles.reshape(n_samples, n_features)
scaler = StandardScaler()
X_temp_angles, X_test_angles, _, _ = train_test_split(flattened_angles, encoded_categories, test_size=0.2, random_state=42)
scaler.fit(X_temp_angles)
X_train_angles_norm = scaler.transform(X_temp_angles)
X_test_angles_norm = scaler.transform(X_test_angles)
X_train_angles = X_train_angles_norm.reshape(-1, padded_angles.shape[1], padded_angles.shape[2])
X_test_angles = X_test_angles_norm.reshape(-1, padded_angles.shape[1], padded_angles.shape[2])

encoded_categories_array = np.array(encoded_categories)
labels_onehot = to_categorical(encoded_categories_array, num_classes=3)

X_train_clips, X_test_clips, X_train_features, X_test_features, y_train_onehot, y_test_onehot = train_test_split(
    padded_clips, features_preprocessed, labels_onehot, test_size=0.2, random_state=42, stratify=encoded_categories)

In [ ]:
reset_random_seeds()

CombinedDataClips=clasification_models.CombinedModel(
    input_shape_clips=X_train_clips.shape[1:],
    input_shape_angles=X_train_angles.shape[1:],
    input_shape_features=X_train_features.shape[1:],
    num_classes=3
)
project_name= 'combined_tuning'
tuner = RandomSearch(
    CombinedDataClips,
    objective='val_accuracy',
    max_trials=10,
    directory='tuner_results',
    project_name= project_name
)

callbacks = [
        TensorBoard(log_dir=os.path.join('logs', project_name)),
        ModelCheckpoint(f'best_model_{project_name}.h5', monitor='val_accuracy', save_best_only=True, verbose=1)
    ]
tuner.search([X_train_clips, X_train_angles, X_train_features], y_train_onehot, epochs=7, batch_size=32, validation_split=0.2, callbacks=callbacks)

best_model = tuner.get_best_models(num_models=1)[0]
best_model.save(f'best_model_{project_name}.h5')

Trial 10 Complete [00h 02m 28s]
val_accuracy: 0.7327188849449158

Best val_accuracy So Far: 0.764976978302002
Total elapsed time: 00h 21m 20s


In [ ]:
loaded_model = load_model("./best_model_combined_tuning.h5")
test_loss, test_accuracy = loaded_model.evaluate([X_test_clips, X_test_angles, X_test_features], y_test_onehot)
print(f"Test Loss: {test_loss}, Test Accuracy: {test_accuracy}")

9/9 [==============================] - 4s 268ms/step - loss: 0.5741 - accuracy: 0.7454
Test Loss: 0.5740699172019958, Test Accuracy: 0.7453874349594116
